In [34]:
import numpy as np
import time
from kinematics import SerialArm
from visualization import VizScene

np.set_printoptions(precision=4, suppress=True)

# --- Define the 6-DOF Robot Arm (from Problem 2) ---
dh_params_6dof = [
    [0,         0.2, 0.0, -np.pi/2],
    [0,         0,   0.2,  0],
    [np.pi/2,   0,   0,    np.pi/2],
    [np.pi/2,   0.4, 0,   -np.pi/2],
    [0,         0,   0,    np.pi/2],
    [0,         0.4, 0,    0]
]
joint_types_6dof = ['r', 'r', 'r', 'r', 'r', 'r']
arm = SerialArm(dh_params_6dof, joint_types_6dof)

# --- Define Target Positions and Initial Configurations from HW PDF ---
targets = [
    np.array([-0.149, 0.364, 1.03]),
    np.array([-0.171, -0.682, -0.192]),
    np.array([0.822, -0.1878, 0.533]),
    np.array([-0.336, 0.095, 0.931]),
    np.array([0.335, 0.368, 0.88])
]

q_starts = {
    "q1 (all zeros)": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "q2 (pi/4)": np.array([np.pi/4] * 6)
}

# Define a gain matrix K for the IK solver
K_gain = np.eye(3) * 0.1

In [35]:
# Cell 2: Run IK Solvers (BUG FIXED)

print(f"{'Target':<8} {'Start Config':<16} {'Method':<6} {'Converged?':<12} {'Iterations':<12} {'Final Error':<15}")
print("="*80)

# Store results for later comparison
results = {}

for i, target in enumerate(targets):
    results[f"Target {i+1}"] = {}
    for q_name, q0 in q_starts.items():
        results[f"Target {i+1}"][q_name] = {}
        
        # --- i. Damped Pseudo-Inverse Method ---
        qf_pinv, err_pinv, it_pinv, conv_pinv = arm.ik_position(
            target, q0, method='pinv', K=K_gain, max_iter=200, tol=1e-3
        )
        # BUG FIX: Save a .copy() of the result array
        results[f"Target {i+1}"][q_name]['pinv'] = qf_pinv.copy()
        print(f"Target {i+1:<1} {q_name:<16} {'pinv':<6} {str(conv_pinv):<12} {it_pinv:<12} {np.linalg.norm(err_pinv):<15.5f}")

        # --- ii. Jacobian Transpose Method ---
        qf_jt, err_jt, it_jt, conv_jt = arm.ik_position(
            target, q0, method='J_T', K=K_gain, max_iter=1000, tol=1e-3
        )
        # BUG FIX: Save a .copy() of the result array
        results[f"Target {i+1}"][q_name]['J_T'] = qf_jt.copy()
        print(f"Target {i+1:<1} {q_name:<16} {'J_T':<6} {str(conv_jt):<12} {it_jt:<12} {np.linalg.norm(err_jt):<15.5f}")
    print("-"*80)

print("\nIK calculations complete. Results are now stored correctly.")

Target   Start Config     Method Converged?   Iterations   Final Error    
Target 1 q1 (all zeros)   pinv   True         70           0.00095        
Target 1 q1 (all zeros)   J_T    True         0            0.00095        
Target 1 q2 (pi/4)        pinv   True         71           0.00093        
Target 1 q2 (pi/4)        J_T    True         0            0.00093        
--------------------------------------------------------------------------------
Target 2 q1 (all zeros)   pinv   True         71           0.00096        
Target 2 q1 (all zeros)   J_T    True         0            0.00096        
Target 2 q2 (pi/4)        pinv   True         71           0.00093        
Target 2 q2 (pi/4)        J_T    True         0            0.00093        
--------------------------------------------------------------------------------
Target 3 q1 (all zeros)   pinv   True         69           0.00093        
Target 3 q1 (all zeros)   J_T    True         0            0.00093        
Target 3 q2 (

In [43]:
# Cell 4: Visualization for the Homework Report (Focusing on Target 5)

# --- Select Target 5, as it clearly shows two different solutions ---
target_idx = 3 # Corresponds to Target 5 from the list (0-indexed)
target_to_viz = targets[target_idx]
target_name = f"Target {target_idx + 1}"

# --- Retrieve the 'pinv' solutions from the two DIFFERENT starting points ---
# These will be visibly different for Target 5
qf_from_q1 = results[target_name]["q1 (all zeros)"]['pinv']
qf_from_q2 = results[target_name]["q2 (pi/4)"]['pinv']

print(f"--- Visualizing 'pinv' solutions for {target_name} ---")
print("This target is chosen because it produces two visibly different final postures.")
print(f"\nFinal q starting from q1 (all zeros):\n{qf_from_q1}")
print(f"\nFinal q starting from q2 (all pi/4):\n{qf_from_q2}")

# --- Set up the visualization scene ---
viz = VizScene()

# Add a single red marker for the target position
viz.add_marker(target_to_viz, color=[1, 0, 0, 1], radius=0.05)

# Add two arms to the scene to compare their final poses
viz.add_arm(arm, joint_colors=[[0.1, 0.2, 0.8, 1]]*6) # Blue arm
viz.add_arm(arm, joint_colors=[[0, 0.6, 0.2, 1]]*6)   # Green arm

# Update the arms to their respective final configurations
viz.update(qs=[qf_from_q1, qf_from_q2])

print("\n--- Plot Description ---")
print("Blue arm = Solution when starting from q1 (all zeros).")
print("Green arm = Solution when starting from q2 (all pi/4).")
print("\nAs you observed, for this specific target, the different starting points")
print("lead to two unique final postures, clearly demonstrating robot redundancy.")
viz.hold()

--- Visualizing 'pinv' solutions for Target 4 ---
This target is chosen because it produces two visibly different final postures.

Final q starting from q1 (all zeros):
[ -0.6302   9.9775 -12.3772  -1.8987   7.5017  -0.    ]

Final q starting from q2 (all pi/4):
[ 2.5604 -1.8204  1.4337  1.2824  1.1706  0.7854]

--- Plot Description ---
Blue arm = Solution when starting from q1 (all zeros).
Green arm = Solution when starting from q2 (all pi/4).

As you observed, for this specific target, the different starting points
lead to two unique final postures, clearly demonstrating robot redundancy.
